<a href="https://colab.research.google.com/github/whkwls2653/Kaggle_JaneStreet/blob/main/colab_TPU_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
if 'train' in globals():
  print("there")
else:
  print('no')

no


In [21]:
########################################여기 참조하세요!!!!!!!!!!!!!!#####################
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)
########################################여기 참조하세요!!!!!!!!!!!!!!#####################

INFO:tensorflow:Initializing the TPU system: grpc://10.119.239.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.119.239.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import os

SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)


save=True
resume=False
print("reading csv input .....")
if 'train' in locals():
  print("Alreday have, skip reading")
else:
  train = pd.read_csv('/content/gdrive/MyDrive/Kaggle/janestreet/train.csv')
  
########################################여기 참조하세요!!!!!!!!!!!!!!#####################

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
AUTO = tf.data.experimental.AUTOTUNE



batch_size = 16 * tpu_strategy.num_replicas_in_sync #TPU batch size에 대해서는 잘 모르겠습니다. 기존 batch size였던 5000으로 사용중입니다(여기서 initialization 되나 아래에서 overwritten 됨.)
########################################여기 참조하세요!!!!!!!!!!!!!!#####################




train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
print("finish reading input")


train.fillna(train.mean(),inplace=True)

# action값을 resp가 0이상 -> 가격이 올랐을때 1로 새로운 열을 만들어 채워 넣음

train['action'] = ((train['resp'].values) > 0).astype(int) * ((train['feature_0'].values)==-1).astype(int)

# train['action'] = (train[featrues[0]==1).astype(int)

features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T



# with tf.device('/device:GPU:0'):


def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)

        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model



batch_size = 5000
hidden_units = [145, 145, 145]
dropout_rates = [0.20, 0.20, 0.20, 0.20]
label_smoothing = 1e-2
learning_rate = 1e-3

checkpoint_path = "/content/gdrive/MyDrive/Kaggle/janestreet/simplemlp_feature0_negative1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
if save==True:

  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,monitor='AUC',mode='max', save_best_only=True, verbose=1)





clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )



########################################여기 참조하세요!!!!!!!!!!!!!!#####################
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = clf
model.summary()



if resume== True :
  model.load_weights(checkpoint_path)
else:
  model.fit(X_train, y_train, epochs=200, batch_size=5000,callbacks=[cp_callback] )
########################################여기 참조하세요!!!!!!!!!!!!!!#####################



models = []

models.append(clf)

th = 0.5000


f = np.median
models = models[-3:]
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

reading csv input .....
Alreday have, skip reading
Running on TPU  ['10.119.239.250:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.119.239.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.119.239.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


finish reading input
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 130)]             0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 130)               520       
_________________________________________________________________
dropout_4 (Dropout)          (None, 130)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 145)               18995     
_________________________________________________________________
batch_normalization_5 (Batch (None, 145)               580       
_________________________________________________________________
activation_4 (Activation)    (None, 145)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14